In [1]:
import pandas as pd

In [2]:
# Importamos nuestra base de datos
data = %time pd.read_excel("CIB-1922.xlsx", parse_dates=[1])

# Cambiamos los nombres de las columnas
data.columns = ["Datetime", "Sistema", "Nemo", "Descripcion", "Tasa_ref", "P_limpio", "P_sucio",
                "Tasa", "CantidadCOP", "VolumenCOP", "Moneda", "Rueda"]

data = data.drop_duplicates(subset="Datetime", keep="first")

CPU times: user 30.8 s, sys: 170 ms, total: 31 s
Wall time: 31.1 s


In [3]:
# Vemos cuantas transacciones hay en nuestra base de datos
data.shape

(263981, 12)

In [4]:
def process_data(dat):
    """
    -----------------------------------------------------
    Funcion que filtra los datos: Bonos tasa fija en COP.
    Fija el timestamp como indice
    -----------------------------------------------------
    
    Parametros: dat - Pandas DataFrame
        Datos con las transacciones intradia de los bonos
    
    Return: dat - Pandas DataFrame
        Datos filtrados
    """
    
    # Nos interesan las transacciones del mercado secundario, en COP, tasa fija
    cond = (dat["Sistema"] == "Transaccional") & (dat["Moneda"] == "COP") & (dat["Nemo"].str.contains("TFIT"))
    dat  = dat.loc[cond].copy()
    
    # Inicializamos la columna Dia
    dat['Dia'] = pd.to_datetime(dat['Datetime'], format='%Y:%M:%D').dt.date
    
    # Indexamos los datos a partir de la columna Datetime
    dat.set_index("Datetime", inplace=True)
    dat = dat.between_time('9:30','15:40')
    
    return dat

In [5]:
# Aplicamos el procesamiento
data = process_data(data)

In [6]:
# Vemos cuantas transacciones nos quedan en la base de datos
data.shape

(121992, 12)

In [7]:
data.head()

,Sistema,Nemo,Descripcion,Tasa_ref,P_limpio,P_sucio,Tasa,CantidadCOP,VolumenCOP,Moneda,Rueda,Dia
Datetime,,,,,,,,,,,,
2017-01-02 09:42:39,Transaccional,TFIT06211118,TITULOS TES,FS,97.952,98.527,6.175,5000000000,4.926365e+09,COP,SPOT,2017-01-02
2017-01-02 09:42:44,Transaccional,TFIT06211118,TITULOS TES,FS,97.948,98.523,6.177,5000000000,4.926165e+09,COP,SPOT,2017-01-02
2017-01-02 09:44:55,Transaccional,TFIT06211118,TITULOS TES,FS,97.948,98.523,6.177,4000000000,3.940932e+09,COP,SPOT,2017-01-02
2017-01-02 09:44:59,Transaccional,TFIT06211118,TITULOS TES,FS,97.948,98.523,6.177,4000000000,3.940932e+09,COP,SPOT,2017-01-02
2017-01-02 09:52:23,Transaccional,TFIT06211118,TITULOS TES,FS,97.950,98.525,6.176,4000000000,3.941012e+09,COP,SPOT,2017-01-02


In [8]:
data[["Dia", "Nemo", "P_limpio", "P_sucio", 
      "Tasa", "CantidadCOP", "VolumenCOP"]].to_csv("prep_data.csv")